In [1]:
import warnings
warnings.filterwarnings('ignore')
import nvtabular as nvt
import cudf
import config
import os
# ignore warnings
from merlin.core.dispatch import get_lib
from merlin.schema.tags import Tags

import time
import merlin.models.tf as mm
from tensorflow.keras.callbacks import EarlyStopping

from merlin.schema.tags import Tags
from nvtabular.ops import *

from merlin.schema.tags import Tags
from merlin.io.dataset import Dataset
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np

2023-10-04 08:29:51.752796: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-04 08:29:52.026135: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2023-10-04 08:29:54.304579: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-04 08:29:54.304808: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-04 08:29:54.304860: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/s

[INFO]: sparse_operation_kit is imported
[SOK INFO] Import /usr/local/lib/python3.8/dist-packages/merlin_sok-1.2.0-py3.8-linux-x86_64.egg/sparse_operation_kit/lib/libsok_experiment.so
[SOK INFO] Import /usr/local/lib/python3.8/dist-packages/merlin_sok-1.2.0-py3.8-linux-x86_64.egg/sparse_operation_kit/lib/libsok_experiment.so
[SOK INFO] Initialize finished, communication tool: horovod


2023-10-04 08:29:55.015710: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-04 08:29:55.015831: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-04 08:29:55.015881: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/s

In [2]:
train = nvt.Dataset(os.path.join(config.data_processed_dir,'train.parquet'),engine="parquet",  part_mem_fraction=0.1)
# valid = nvt.Dataset(os.path.join(config.data_processed_dir,'valid.parquet'),engine="parquet", part_mem_fraction=0.1)

In [3]:
# customer_features = ['customer_id', 'age', 'FN', 'Active', 'club_member_status','fashion_news_frequency']
customer_features = ['customer_id', 'age']
article_features = ['article_id', 'product_code', 'product_type_no', 'graphical_appearance_no', 'colour_group_code', 'perceived_colour_value_id', 'perceived_colour_master_id', 'department_no', 'index_code', 'index_group_no', 'section_no', 'garment_group_no']

In [4]:
articles = cudf.read_parquet(os.path.join(config.data_raw_dir,'articles.parquet'))[article_features]
customers = cudf.read_parquet(os.path.join(config.data_processed_dir,'customers_enc.parquet'))[customer_features]


In [5]:
articles = nvt.Dataset(articles)
customers = nvt.Dataset(customers)

In [6]:
train = Dataset.merge(train, articles, on="article_id", how="inner")
train = Dataset.merge(train, customers, on="customer_id", how="inner")
train.to_parquet(os.path.join(config.data_processed_dir,'transformer4rec', 'train.parquet'))
# valid = Dataset.merge(valid, articles, on="article_id", how="inner")
# valid = Dataset.merge(valid, customers, on="customer_id", how="inner")
# valid.to_parquet(os.path.join(config.data_processed_dir,'transformer4rec', 'valid.parquet'))
# print(len(train.to_ddf()), len(valid.to_ddf()))
# del articles, customers, train, valid
print(len(train.to_ddf()))
del articles, customers, train

7516884


In [7]:
# ['t_dat', 'customer_id', 'article_id', 'price', 'sales_channel_id','week']
article_id_old = ["article_id"] >> Categorify() >> TagAsItemFeatures()
article_id = ["article_id"] >> Categorify() 
customer_id = ["customer_id"] >> Categorify() >> TagAsUserID()
price = ["price"] >> LogOp() >> Normalize() >> TagAsItemFeatures() 
week_split = ["week"] 
time_features = ["t_dat"]
session_time = (
    time_features >> 
    nvt.ops.LambdaOp(lambda col: cudf.to_datetime(col, unit='s')) >> 
    nvt.ops.Rename(name = 'event_time_dt')
)

day = (session_time
        >> nvt.ops.LambdaOp(lambda col: col.dt.day)
        >> nvt.ops.Rename(name = 'day')
        >> Categorify()
        >> TagAsUserFeatures())
week = (session_time
        >> nvt.ops.LambdaOp(lambda col: col.dt.weekday)
        >> nvt.ops.Rename(name = 'week_day')
        >> Categorify()
        >> TagAsUserFeatures())
month = (session_time
        >> nvt.ops.LambdaOp(lambda col: col.dt.month)
        >> nvt.ops.Rename(name = 'month')
        >> Categorify()
        >> TagAsUserFeatures())
year = (session_time
        >> nvt.ops.LambdaOp(lambda col: col.dt.year)
        >> nvt.ops.Rename(name = 'year')
        >> Categorify()
        >> TagAsUserFeatures())

In [8]:
age =  ['age'] >>  LogOp() >> Normalize() >> TagAsUserFeatures() >> AddTags([Tags.USER,Tags.CONTINUOUS])
# cat_customer_columns = (['FN', 'Active', 'club_member_status','fashion_news_frequency'] 
#             >> Categorify() 
#             >> TagAsUserFeatures()
#             )

In [9]:
cat_article_columns = (article_features 
              >> Categorify()  
              >> TagAsItemFeatures())

In [10]:
SESSIONS_MAX_LENGTH =30
SESSIONS_MIN_LENGTH = 3

filtered_articles = (article_id + customer_id + ['t_dat'] + price + day + week + month + year + age  + cat_article_columns + week_split
                      >> Filter(f=lambda df: df["article_id"]!=0)
)
# filtered_articles  = article_id + customer_id + ['t_dat'] + price + day + week + month + year + age  + cat_article_columns + week_split
groupby_features = (filtered_articles
                    >> Groupby(
                        groupby_cols=['customer_id', 'week' ],
                        aggs={
                            'article_id': ['list', 'count'],
                            'price': ['list','mean', 'std', 'max'],
                            'product_code': ['list'],
                            'product_type_no': ['list'],
                            'graphical_appearance_no': ['list'],
                            'colour_group_code': ['list'],
                            'perceived_colour_value_id': ['list'],
                            'perceived_colour_master_id': ['list'],
                            'department_no': ['list'],
                            'index_code': ['list'],
                            'index_group_no': ['list'],
                            'section_no': ['list'],
                            'garment_group_no': ['list'],
                            
                            'day': ['list'],
                            'week_day': ['list'],
                            'month': ['list'],
                            'year': ['list'],
                        },
                        sort_cols=["t_dat"],
                        name_sep="_"
                    )
                )


In [11]:
list_columns = ['price_list',
 'day_list',
 'week_day_list',
 'month_list',
 'year_list',
 'product_code_list',
 'product_type_no_list',
 'graphical_appearance_no_list',
 'colour_group_code_list',
 'perceived_colour_value_id_list',
 'perceived_colour_master_id_list',
 'department_no_list',
 'index_code_list',
 'index_group_no_list',
 'section_no_list',
 'garment_group_no_list']

groupby_features_articles_id= (groupby_features['article_id_list'] 
                         >> AddTags([Tags.ITEM, Tags.ITEM_ID, Tags.SEQUENCE])
                        )
groupby_features_articles_features= (groupby_features[list_columns]
                         >> AddTags([Tags.ITEM, Tags.SEQUENCE])
                        )
truncated_features = (groupby_features_articles_id + groupby_features_articles_features
                      >> ListSlice(-SESSIONS_MAX_LENGTH) 
                     )
stat_features = (groupby_features['article_id_count','price_max','price_mean','price_std']
               >> LogOp()
               >> Normalize()
               >> Rename(postfix="_norm")
               >> AddTags([Tags.ITEM,Tags.CONTINUOUS])
              )

output = (groupby_features['customer_id',"article_id_count","week"] + truncated_features + stat_features 
                     >> Filter(f=lambda df: df["article_id_count"] >= SESSIONS_MIN_LENGTH)
                    )

In [12]:
train = nvt.Dataset(os.path.join(config.data_processed_dir,'transformer4rec', 'train.parquet'),engine="parquet",)
# valid = nvt.Dataset(os.path.join(config.data_processed_dir,'transformer4rec', 'valid.parquet'),engine="parquet")
# print(len(train.to_ddf()), len(valid.to_ddf()))
print(len(train.to_ddf()))
wf = nvt.Workflow(output)
print(train.to_ddf().week.value_counts().compute())

7516884
91     549443
92     517428
81     401878
87     387772
88     363092
93     337419
97     319469
89     310996
86     300024
96     299469
90     298890
83     297896
95     289528
98     289161
94     286672
101    283181
99     265211
102    264657
85     259586
100    256644
103    255241
104    240311
82     229433
84     213483
Name: week, dtype: int64


In [13]:
train = wf.fit_transform(train)
# valid = wf.transform(valid)
# print(len(train.to_ddf()), len(valid.to_ddf()))
print(len(train.to_ddf()))
# train.regenerate_dataset(os.path.join(config.data_final,'transformer4rec', 'train.parquet'),part_size="500MiB", file_size="500MiB")
# valid.regenerate_dataset(os.path.join(config.data_final,'transformer4rec', 'valid.parquet'),part_size="500MiB", file_size="500MiB")

1016403


In [14]:
train.to_ddf()

,customer_id,article_id_count,week,article_id_list,price_list,day_list,week_day_list,month_list,year_list,product_code_list,product_type_no_list,graphical_appearance_no_list,colour_group_code_list,perceived_colour_value_id_list,perceived_colour_master_id_list,department_no_list,index_code_list,index_group_no_list,section_no_list,garment_group_no_list,article_id_count_norm,price_max_norm,price_mean_norm,price_std_norm
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,,
,int64,int32,int64,int64,float64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [15]:
train.to_ddf()['week'].value_counts().compute()

91     72974
92     69187
87     52643
88     50132
81     49541
93     46306
97     43776
89     42785
96     41842
90     41305
86     40514
95     40031
98     39866
94     39524
101    39278
83     38777
102    36320
99     35962
100    35167
85     34751
103    34667
104    32604
82     30110
84     28341
Name: week, dtype: int64

In [16]:
train.to_parquet(os.path.join(config.data_final,'transformer4rec', 'train.parquet'))
# valid.to_parquet(os.path.join(config.data_final,'transformer4rec', 'valid.parquet'))
wf.save(os.path.join(config.data_final,'transformer4rec', 'workflow'))

In [21]:
train.to_ddf().head()   

,customer_id,article_id_count,week,article_id_list,price_list,day_list,week_day_list,month_list,year_list,product_code_list,...,perceived_colour_master_id_list,department_no_list,index_code_list,index_group_no_list,section_no_list,garment_group_no_list,article_id_count_norm,price_max_norm,price_mean_norm,price_std_norm
0,1,14,83,"[13, 471, 471, 471, 471, 151, 157, 623, 133, 1...","[-0.5834894776344299, 0.23546873033046722, 0.2...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]","[7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]","[3, 32, 32, 32, 32, 389, 157, 3, 58, 58, 25, 2...",...,"[9, 12, 12, 12, 12, 3, 9, 12, 4, 4, 4, 4, 3, 9]","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]",2.170507,0.051563,0.067199,-0.295358
1,2,8,82,"[2734, 2734, 703, 703, 12679, 16185, 11025, 5283]","[0.9432718753814697, 0.9432718753814697, 0.438...","[8, 8, 8, 8, 8, 8, 8, 8]","[4, 4, 4, 4, 4, 4, 4, 4]","[7, 7, 7, 7, 7, 7, 7, 7]","[3, 3, 3, 3, 3, 3, 3, 3]","[258, 258, 547, 547, 3566, 3566, 1699, 116]",...,"[5, 5, 10, 10, 3, 10, 7, 6]","[44, 44, 10, 10, 58, 58, 25, 53]","[3, 3, 3, 3, 6, 6, 4, 3]","[3, 3, 3, 3, 5, 5, 4, 3]","[10, 10, 3, 3, 14, 14, 5, 7]","[7, 7, 8, 8, 4, 4, 8, 15]",1.314833,0.946309,0.706403,0.407315
2,4,3,89,"[5776, 2896, 4178]","[1.4439300298690796, 0.438526451587677, 0.9432...","[26, 26, 26]","[3, 3, 3]","[3, 3, 3]","[3, 3, 3]","[3392, 2086, 2987]",...,"[11, 3, 3]","[47, 8, 47]","[5, 5, 5]","[3, 3, 3]","[6, 6, 6]","[6, 6, 6]",-0.043541,0.755427,0.901575,0.022450
3,4,4,99,"[221, 70, 5169, 923]","[0.03174927085638046, 0.438526451587677, 1.244...","[14, 14, 14, 14]","[3, 3, 3, 3]","[6, 6, 6, 6]","[3, 3, 3, 3]","[170, 50, 1973, 40]",...,"[3, 3, 3, 8]","[24, 20, 37, 14]","[3, 8, 3, 8]","[3, 6, 3, 6]","[3, 11, 18, 11]","[12, 3, 3, 3]",0.330243,0.667434,0.523468,0.462210
4,7,9,81,"[12887, 2200, 13127, 43, 23919, 12, 53, 16979,...","[-1.1216965913772583, -0.7899370789527893, -0....","[3, 3, 3, 3, 3, 3, 3, 3, 3]","[4, 4, 4, 4, 4, 4, 4, 4, 4]","[7, 7, 7, 7, 7, 7, 7, 7, 7]","[3, 3, 3, 3, 3, 3, 3, 3, 3]","[5436, 261, 5861, 64, 12738, 17, 64, 8726, 10369]",...,"[8, 4, 3, 4, 3, 3, 3, 8, 3]","[45, 35, 127, 45, 14, 45, 45, 45, 24]","[5, 7, 4, 5, 8, 5, 5, 5, 3]","[3, 3, 4, 3, 6, 3, 3, 3, 3]","[13, 16, 24, 13, 11, 13, 13, 13, 3]","[16, 14, 14, 16, 3, 16, 16, 16, 12]",1.491320,-0.802324,-1.647960,-0.741336


In [17]:
from transformers4rec.utils.data_utils import save_time_based_splits
save_time_based_splits(train,
                       output_dir=os.path.join(config.data_final,'time'),
                       partition_col='week',
                       timestamp_col='customer_id', 
                      )

Creating time-based splits: 100%|██████████| 24/24 [00:05<00:00,  4.24it/s]


In [66]:

path= os.path.join(config.data_raw_dir, 'image_embeddings_norm.parquet')
image_embeddings = pd.read_parquet(path)


In [65]:
articles = pd.read_parquet(os.path.join(config.data_final,'transformer4rec', 'workflow','categories','unique.article_id.parquet'))

In [67]:
image_embeddings['article_id'] = image_embeddings['article_id'].astype('int64')

In [68]:
articles["encoded_article_id"] = articles.index
articles.head()

,article_id,article_id_size,encoded_article_id
3,706016001,10174,3
4,610776002,9997,4
5,610776001,8840,5
6,599580038,8556,6
7,372860002,8384,7


In [69]:
query_embs_df = pd.merge(image_embeddings,
                         articles['encoded_article_id','article_id'], 
                         how="inner",
                         on="article_id")


KeyError: 'article_id'

In [64]:
query_embs_df

,0,1,2,3,4,5,6,7,8,9,...,505,506,507,508,509,510,511,article_id,article_id_size,index
0,0.039765,-0.016198,-0.041136,0.008217,0.096949,-0.052870,-0.007064,-0.019216,0.010876,0.029862,...,-0.020590,0.052057,0.022599,-0.018661,0.024323,-0.017156,0.039991,866501003,13,27767
1,0.004825,0.160624,0.001141,-0.076373,-0.053264,-0.044805,-0.049826,0.019800,0.055599,0.097122,...,0.013052,-0.018503,0.048020,-0.008739,0.005817,-0.016124,0.005485,895582003,1285,1057
2,-0.030920,0.026129,-0.031214,0.022818,-0.008278,-0.014560,-0.007099,0.029508,0.107154,0.084394,...,-0.034007,0.058463,0.031537,0.009217,0.005818,0.001128,-0.004276,835008009,1601,699
3,-0.000500,-0.052893,-0.040797,-0.013157,0.057557,-0.026879,-0.040061,-0.009636,0.008952,0.001990,...,-0.001934,0.063831,0.080052,-0.013869,-0.024339,0.013448,-0.027274,578630016,45,18743
4,0.011811,-0.011214,-0.007311,-0.033261,0.016670,-0.026183,-0.009956,0.011440,0.053363,0.028482,...,-0.019107,0.014246,0.054741,-0.076430,0.026880,0.030014,-0.027249,617660001,2,40266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50142,0.012994,-0.002257,0.063309,-0.010290,0.030534,-0.059509,-0.022469,0.051625,0.070573,0.031021,...,-0.049616,0.043205,0.029583,0.020836,0.009513,-0.017151,0.010349,801768001,303,6523
50143,0.040033,0.155111,0.030523,-0.018981,0.001548,-0.066222,-0.052114,0.020624,0.051297,0.048284,...,-0.052050,0.113082,0.065778,-0.009320,0.022359,0.011899,0.000645,909576002,4,37197
50144,-0.007579,-0.006353,0.033088,0.040989,-0.000415,-0.040543,-0.014090,0.029763,0.093931,0.072243,...,-0.052309,0.057194,0.023288,-0.008273,0.030906,-0.004835,-0.000924,776106001,11,28743
50145,0.037749,0.147308,0.020192,-0.034558,-0.018647,0.004227,0.020621,-0.005747,0.007071,0.136995,...,-0.006387,0.073987,0.018263,0.055379,0.041826,0.021242,0.016452,879995002,92,13920


In [53]:
articles

,article_id,article_id_size
3,706016001,10174
4,610776002,9997
5,610776001,8840
6,599580038,8556
7,372860002,8384
...,...,...
50246,941138001,1
50247,941138004,1
50248,942035001,1
50249,942506001,1


In [44]:
articles

,article_id,article_id_size
3,706016001,10174
4,610776002,9997
5,610776001,8840
6,599580038,8556
7,372860002,8384
...,...,...
50246,941138001,1
50247,941138004,1
50248,942035001,1
50249,942506001,1
